In [ ]:
#! pip install pyarrow

In [ ]:
#! pip install requests

In [ ]:
import pyarrow.parquet as pq
import pandas as pd

In [ ]:
taxi_zones = [4,12,13,45,43,48,50,68,79,87,88,90,125,100,107,113,114,140,137,141,142,143,144,148,158,161,162,163,
              164,170,186,194,209,211,224,229,230,231,239,232,233,234,236,237,238,246,249,261]

    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet"
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet"
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet"
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet"
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-06.parquet"
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-07.parquet"
    

  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-02.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-03.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-04.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-05.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-06.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-07.parquet"


In [ ]:
# URL to get the data
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet"

# Read the Parquet file from the URL
df_raw = pd.read_parquet(url, engine='pyarrow')

In [ ]:
# Separate date and time for pickup and dropoff
df_raw['pickup_date'] = df_raw['tpep_pickup_datetime'].dt.date
df_raw['pickup_time'] = df_raw['tpep_pickup_datetime'].dt.time
df_raw['dropoff_date'] = df_raw['tpep_dropoff_datetime'].dt.date
df_raw['dropoff_time'] = df_raw['tpep_dropoff_datetime'].dt.time

# Calculate trip time in seconds
df_raw['trip_time_minutes'] = (df_raw['tpep_dropoff_datetime'] - df_raw['tpep_pickup_datetime']).dt.total_seconds() / 60

In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3007526 entries, 0 to 3007525
Data columns (total 24 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [ ]:
# Define the custom filter function for Friday 4 PM to Sunday 8 PM
def filter_by_time(row):
    day = row['tpep_pickup_datetime'].dayofweek  # 0: Monday, 6: Sunday
    hour = row['tpep_pickup_datetime'].hour

    # Check for Friday after 4 PM
    if day == 4 and hour >= 16:  # Friday 4 PM onwards
        return True
    # Check for all day Saturday
    elif day == 5:  # All day Saturday
        return True
    # Check for Sunday until 8 PM
    elif day == 6 and hour <= 20:  # Sunday until 8 PM
        return True
    else:
        return False

# Apply the filter to the DataFrame
df_raw = df_raw[df_raw.apply(filter_by_time, axis=1)]

In [ ]:
fees_amount = ['extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'Airport_fee']
df_raw['total_fees'] = df_raw[fees_amount].sum(axis=1)

In [ ]:
selected_columns = ['pickup_date', 'pickup_time', 'dropoff_date', 'dropoff_time', 'PULocationID',
                    'DOLocationID', 'trip_distance', 'trip_time_minutes', 'fare_amount',
                    'total_fees', 'tip_amount', 'total_amount', 'payment_type', 'passenger_count']
new_df = df_raw[selected_columns]

In [ ]:
payment_type_mapping = {
    1: 'Credit card',
    2: 'Cash',
    3: 'No charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided trip'
}

new_df['payment_type'] = new_df['payment_type'].map(payment_type_mapping)

In [ ]:
f_df = new_df[
    new_df['PULocationID'].isin(taxi_zones) &
    new_df['DOLocationID'].isin(taxi_zones)
]

In [ ]:
f_df = f_df.reset_index()

In [ ]:
f_df= f_df.drop(columns=['index'])

In [ ]:
f_df.head()

In [ ]:
f_df.to_csv('ride_sharing2.csv', index=False)

In [ ]:
# Read both CSV files into pandas DataFrames
df2 = pd.read_csv('/content/taxi_data2.csv')
df3 = pd.read_csv('/content/taxi_data3.csv')
df4 = pd.read_csv('/content/taxi_data4.csv')
df5 = pd.read_csv('/content/taxi_data5.csv')
df6 = pd.read_csv('/content/taxi_data6.csv')
df7 = pd.read_csv('/content/taxi_data7.csv')

# Append df2 to df1
taxi_df = pd.concat([df2, df3, df4, df5, df6, df7], ignore_index=True)

# Save the appended DataFrame to a new CSV file
#taxi_df.to_csv('taxi_data.csv', index=False)

<ipython-input-146-79c415f9dd33>:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('/content/taxi_data2.csv')
<ipython-input-146-79c415f9dd33>:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('/content/taxi_data3.csv')
<ipython-input-146-79c415f9dd33>:4: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv('/content/taxi_data4.csv')
<ipython-input-146-79c415f9dd33>:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df5 = pd.read_csv('/content/taxi_data5.csv')
<ipython-input-146-79c415f9dd33>:6: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df6 = pd.read_csv('/content/taxi_data6.csv')
<ipython-input-146-79c415f9dd33>:7: DtypeWarning: Columns (12) have mixed types.

In [ ]:
taxi_df = taxi_df.dropna()

In [ ]:
taxi_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4301055 entries, 0 to 4820927
Data columns (total 14 columns):
 #   Column             Dtype  
---  ------             -----  
 0   pickup_date        object 
 1   pickup_time        object 
 2   dropoff_date       object 
 3   dropoff_time       object 
 4   PULocationID       int64  
 5   DOLocationID       int64  
 6   trip_distance      float64
 7   trip_time_minutes  float64
 8   fare_amount        float64
 9   total_fees         float64
 10  tip_amount         float64
 11  total_amount       float64
 12  payment_type       object 
 13  passenger_count    float64
dtypes: float64(7), int64(2), object(5)
memory usage: 492.2+ MB


In [ ]:
# Randomly sample n rows from the dataframe
n = 60000  # specify the number of rows to sample
sample_taxi_df = taxi_df.sample(n)

In [ ]:
sample_taxi_df.head()

,pickup_date,pickup_time,dropoff_date,dropoff_time,PULocationID,DOLocationID,trip_distance,trip_time_minutes,fare_amount,total_fees,tip_amount,total_amount,payment_type,passenger_count
3442744,2024-06-01,22:58:38,2024-06-01,23:03:07,231,249,0.00,4.483333,5.8,5.0,3.24,14.04,Credit card,1.0
3239979,2024-05-31,19:05:31,2024-05-31,19:15:25,79,4,1.07,9.900000,10.7,6.5,0.00,17.20,Credit card,2.0
2986153,2024-05-18,15:59:20,2024-05-18,16:21:27,148,170,1.91,22.116667,19.8,4.0,0.00,23.80,Credit card,2.0
1513480,2024-03-30,20:33:04,2024-03-30,20:44:41,246,50,0.95,11.616667,11.4,5.0,3.28,19.68,Credit card,1.0
2573770,2024-05-04,01:14:19,2024-05-04,01:23:42,114,107,1.51,9.383333,10.7,5.0,3.00,18.70,Credit card,2.0


In [ ]:
sample_taxi_df.to_csv('taxi_data.csv', index=False)